In [1]:
def obj_recog():
    orig_img_path = '/home/xilinx/jupyter_notebooks/common/data/webcam.jpg'
    !fswebcam  --no-banner --save {orig_img_path} -d /dev/video0 2
    # jupyter调用摄像头的代码
    im = cv2.imread(orig_img_path)
    #cv2.imshow('t',im)
    #cv2.imgshow(im)
    width = im.shape[1]
    height = im.shape[0]
    rate = round(width*0.5)
    rate1 = round(height*0)
    half = round(height/2)
    ims = im[rate1:half,0:(width-rate)]
    ims_right = im[(half+1):(height-rate1),0:(width-rate)]
    lower_red = np.array([0, 43, 20]) # 红色范围低阈值
    upper_red = np.array([20, 255, 255]) # 红色范围高阈值
    lower_red2 = np.array([150, 43, 20]) # 红色范围低阈值
    upper_red2 = np.array([200, 255, 255]) # 红色范围高阈值
    hsv_img = cv2.cvtColor(ims, cv2.COLOR_BGR2HSV)
    mask_red = cv2.inRange(hsv_img, lower_red, upper_red)
    mask_red2 = cv2.inRange(hsv_img, lower_red2, upper_red2)
    mask = cv2.bitwise_or(mask_red,mask_red2)
    # again for right side
    hsv_img_right = cv2.cvtColor(ims_right, cv2.COLOR_BGR2HSV)
    mask_red_right = cv2.inRange(hsv_img_right, lower_red, upper_red)
    mask_red_right2 = cv2.inRange(hsv_img_right, lower_red2, upper_red2)
    mask_right = cv2.bitwise_or(mask_red_right,mask_red_right2)
    
    #print(mask_red)
    #print(mask_red2)
    #print(mask)
    #print(np.sum(mask_red))
    #print(np.sum(mask_red2))
    #print(np.sum(mask))
    nl = np.sum(mask)
    nr = np.sum(mask_right)
    #print(number)
    number = nl + nr
    
    h1 = height - rate1*2
    w1 = width - rate
    S = h1*w1
    print("nr",nr/255)
    print("nl",nl/255)
    test = nr/255 - nl/255
    print(test )
    print(number/S/255)
    number = (number/S/255)
    
    yu = 2000
    
    if(number < 0.01):
        kind =  0
    else:
        kind = 1
    #if (number>0.01 && number<0.03 && number>0.5):
        #kind = 

    return kind

# des = 
# anti = 

In [2]:
import numpy as np
import cv2
import time
from PIL import Image
np.set_printoptions(threshold=np.inf)

In [3]:
import binascii
import serial
import os
os.system('sh ./stop_sys_ttyPS0.sh')
def run_action(cmd):
    ser = serial.Serial("/dev/ttyPS0", 9600, timeout=5)
    cnt_err = 0
    while 1:
        test_read = ser.read()
        #print('test_read', test_read)
        cnt_err += 1
        if test_read== b'\xa3' or cnt_err == 50:
            break
    
    if cnt_err == 50:
        print('can not get REQ')
    else:
        print('read REQ finished!')
        ser.write(cmd2data(cmd))
        print('send action ok!')
    ser.close()
    
def crc_calculate(package):
    crc = 0
    for hex_data in package:

        b2 = hex_data.to_bytes(1, byteorder='little')
        crc = binascii.crc_hqx(b2, crc)

    return [(crc >> 8), (crc & 255)]    # 校验位两位

def cmd2data(cmd):
    cnt=0
    cmd_list=[]
    for i in cmd:
        cnt+=1
        cmd_list+=[ord(i)]
    cmd_list=[0xff,0xff]+[(cnt+5)>>8,(cnt+5)&255]+[0x01,(cnt+1)&255,0x03]+cmd_list
    cmd_list=cmd_list+crc_calculate(cmd_list)
    return cmd_list

def wait_req():
    ser = serial.Serial("/dev/ttyPS0", 9600, timeout=5)
    while 1:
        test_read=ser.read()
        if test_read== b'\xa3' :
            print('read REQ finished!') 
            break

In [4]:
import cv2 as cv
from PIL import Image as PIL_Image
import numpy as np
import math
from numpy.lib.function_base import disp
def houfline(img):
    #
    #cv.imshow("img",img)
    # cv.waitKey(0)
    # cv.destroyAllWindows()

    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

    l_blue = np.array([[100,43,46]])
    h_blue = np.array([[124,255,255]])
    mask = cv.inRange(hsv, l_blue, h_blue)
    #res = cv.bitwise_and(img, img, mask = mask)
    # cv.imshow("img", img)
    #cv.imshow("mask", mask)
    # #cv.imshow("res", res)
    edges = cv.Canny(mask,50,150,apertureSize = 3)
    lines = cv.HoughLinesP(edges,1,np.pi/180,100,minLineLength=100,maxLineGap=10)
    if lines is None:
        return 0,0,2
    x1,y1,x2,y2 = lines[0][0]
    cv.line(img,(x1,y1),(x2,y2),(0,255,0),2)
    cv.imwrite("/home/xilinx/jupyter_notebooks/common/data/webcam.jpg",img)
    print(x1,y1,x2,y2)
    #shape (480, 640, 3)
    p = [0,240]
    b = y1-y2
    a = x1-x2
    c = -x1*y2 + y1*x2
    m = b/a
    #dis
    #print(a,b)
    dis = abs(a*p[1]+b*p[0]+c)/math.sqrt(a*a+b*b)
    #cv.line((x1,y1),(x2,y2),())
    flag = 0
    if (y1+2)/2 >240:
        flag = 1
    else:
        flag = 0
        
    return dis, m, flag

# img = cv.imread('/home/xilinx/jupyter_notebooks/common/data/bianyuan/0.jpg')
# [dis,m] = houfline(img)
# print(dis,m)

In [5]:
def takepic():
    orig_img_path = '/home/xilinx/jupyter_notebooks/common/data/webcam.jpg'
    !fswebcam  --no-banner --save {orig_img_path} -d /dev/video0 2

In [6]:
def halfrun():
    dis_left = 200
    k_left = 0.5
    dis_right=200
    k_right = -0.5
    mk_min = 10
    for i in range(1,5):
        run_action('fastForward03')
        wait_req()
        takepic()
        img = cv.imread('/home/xilinx/jupyter_notebooks/common/data/webcam.jpg')
        [lk,lr,mk,mr,rk,rr] = houfline(img)
        print('take picture',dis,k,flag)
        while(abs(mk)<mk_min):
            if(mk>0):
                #left
                run_action('turn001L')
                
            elif()    
            
            
            run_action('turn001R')
            wait_req()
            run_action('Right3move')
            wait_req()
        elif(dis<dis_right and k < k_right and flag == 1):
            run_action('turn001L')
            wait_req()
            run_action('Left3move')
            wait_req()
        wait_req()    


def lefty():
    for i in range(12):
        run_action('Left3move') 
        wait_req()
    kind = obj_recog()
    while(kind==1):
        run_action('Left3move')
        wait_req()
        kind = obj_recog() 

def righty():
    for i in range(12):
        run_action('Right3move')
        wait_req()       
    kind = obj_recog()
    while(kind==1):
        run_action('Right3move')
        wait_req()
        kind = obj_recog() 

In [8]:
# des = 1, 2            anti = 1,2,3,4
des = 2
anti = 3

if( des== 1 and anti == 1):
    # right forward2
    righty()
    #
    halfrun()
    halfrun()
    #
    wait_req()

elif(des== 1 and anti == 2):
    # you qian1 zuo qian1 you
    # right forward2
    righty()
    #
    #
    halfrun()
    lefty()

    halfrun()
    #
    righty()
    wait_req()

elif(des== 1 and anti == 3):
    # qian you qian
    halfrun()
    righty()
    halfrun()

elif(des== 1 and anti == 4):
    # qian2 you
    halfrun()
    halfrun()
    righty()
elif(des== 2 and anti == 1):
    # qian2 zuo
    halfrun()
    halfrun()
    lefty()
elif(des== 2 and anti == 2):
    # qian zuo qian
    halfrun()
    lefty()
    halfrun()

elif(des== 2 and anti == 3):
    # zuo qian you qian zuo 
    lefty()
    halfrun()
    righty()
    halfrun()
    lefty()

elif(des== 2 and anti == 4):
    # zuo qian
    lefty()
    halfrun()
    halfrun()

read REQ finished!
send action ok!
read REQ finished!
read REQ finished!
send action ok!
read REQ finished!
read REQ finished!
send action ok!
read REQ finished!
read REQ finished!
send action ok!
read REQ finished!
read REQ finished!
send action ok!
read REQ finished!
read REQ finished!
send action ok!
read REQ finished!
read REQ finished!
send action ok!
read REQ finished!
read REQ finished!
send action ok!
read REQ finished!
read REQ finished!
send action ok!
read REQ finished!
read REQ finished!
send action ok!
read REQ finished!
read REQ finished!
send action ok!
read REQ finished!
read REQ finished!
send action ok!
read REQ finished!
--- Opening /dev/video0...
Trying source module v4l2...
/dev/video0 opened.
No input was specified, using the first.
Adjusting resolution from 384x288 to 640x480.
--- Capturing frame...
Captured frame in 0.00 seconds.
--- Processing captured image...
Disabling banner.
Writing JPEG image to '/home/xilinx/jupyter_notebooks/common/data/webcam.jpg'.
Writ

KeyboardInterrupt: 